# 3-1 Low-level API: Demonstration

The examples below use low-level APIs in TensorFlow to implement a linear regression model.

Low-level API includes tensor operator, graph and automatic differentiates.

In [1]:
import tensorflow as tf

# Print time stamps
@tf.function
def printbar():
    ts = tf.timestamp()
    today_ts = ts%(24*60*60)

    hour = tf.cast(today_ts//3600+8,tf.int32)%tf.constant(24)
    minite = tf.cast((today_ts%3600)//60,tf.int32)
    second = tf.cast(tf.floor(today_ts%60),tf.int32)
    
    def timeformat(m):
        if tf.strings.length(tf.strings.format("{}",m))==1:
            return(tf.strings.format("0{}",m))
        else:
            return(tf.strings.format("{}",m))
    
    timestring = tf.strings.join([timeformat(hour),timeformat(minite),
                timeformat(second)],separator = ":")
    tf.print("=========="*8,end = "")
    tf.print(timestring)

In [2]:
# Number of samples
n = 400

# Generate testing dataset
X = tf.random.uniform([n,2],minval=-10,maxval=10) 
w0 = tf.constant([[2.0],[-1.0]])
b0 = tf.constant(3.0)
Y = X@w0 + b0 + tf.random.normal([n,1],mean = 0.0,stddev= 2.0)  # @ represents matrix multiplication; add Gaussian noise

In [3]:
# Debug in dynamic graph

w = tf.Variable(tf.random.normal(w0.shape))
b = tf.Variable(0.0)

def train(epoches):
    for epoch in tf.range(1,epoches+1):
        with tf.GradientTape() as tape:
            # Loss in forward propagation
            Y_hat = X@w + b
            loss = tf.squeeze(tf.transpose(Y-Y_hat)@(Y-Y_hat))/(2.0*n)   

        # Gradient by back propagation
        dloss_dw,dloss_db = tape.gradient(loss,[w,b])
        # Update parameters using gradient descent method
        w.assign(w - 0.001*dloss_dw)
        b.assign(b - 0.001*dloss_db)
        if epoch%1000 == 0:
            printbar()
            tf.print("epoch =",epoch," loss =",loss,)
            tf.print("w =",w)
            tf.print("b =",b)
            tf.print("")
            
train(5000)

================================================================================16:27:30
epoch = 1000  loss = 2.60077739
w = [[1.96938241]
 [-0.983276486]]
b = 1.86474025

================================================================================16:27:31
epoch = 2000  loss = 2.08383226
w = [[1.9742254]
 [-0.983455658]]
b = 2.5554719

================================================================================16:27:33
epoch = 3000  loss = 2.01372981
w = [[1.97600842]
 [-0.983521402]]
b = 2.809834

================================================================================16:27:34
epoch = 4000  loss = 2.00422359
w = [[1.9766649]
 [-0.983544946]]
b = 2.90350342

================================================================================16:27:35
epoch = 5000  loss = 2.00293422
w = [[1.97690666]
 [-0.983553588]]
b = 2.9379971



In [4]:
## Accelerate the execution by converting the dynamic graph to static using Autograph

w = tf.Variable(tf.random.normal(w0.shape))
b = tf.Variable(0.0)

@tf.function
def train(epoches):
    for epoch in tf.range(1,epoches+1):
        with tf.GradientTape() as tape:
            # Loss in forward propagation
            Y_hat = X@w + b
            loss = tf.squeeze(tf.transpose(Y-Y_hat)@(Y-Y_hat))/(2.0*n)   

        # Gradient by back propagation
        dloss_dw,dloss_db = tape.gradient(loss,[w,b])
        # Update parameters using gradient descent method
        w.assign(w - 0.001*dloss_dw)
        b.assign(b - 0.001*dloss_db)
        if epoch%1000 == 0:
            printbar()
            tf.print("epoch =",epoch," loss =",loss,)
            tf.print("w =",w)
            tf.print("b =",b)
            tf.print("")
train(5000)

================================================================================16:27:38
epoch = 1000  loss = 2.60776782
w = [[1.9693377]
 [-0.983274758]]
b = 1.85836804

================================================================================16:27:38
epoch = 2000  loss = 2.08478
w = [[1.97420871]
 [-0.983455]]
b = 2.55312562

================================================================================16:27:38
epoch = 3000  loss = 2.01385856
w = [[1.97600222]
 [-0.983521163]]
b = 2.80897

================================================================================16:27:38
epoch = 4000  loss = 2.00424099
w = [[1.97666264]
 [-0.983544886]]
b = 2.90318608

================================================================================16:27:38
epoch = 5000  loss = 2.0029366
w = [[1.97690582]
 [-0.983553588]]
b = 2.93788052

